# Eviroment and Board Setup

The cells in this section setup the board from loading the overlay, and the notebook eviroment but loading required librarys.
They should be ran once, before running any other cells

In [1]:
from pynq.overlay import Overlay
neo_overlay = Overlay(r"/home/xilinx/neorv32_overlays/rv32i_overlay.bit")

from pynq import GPIO

neo_reset_n = GPIO(GPIO.get_gpio_pin(0), 'out')
neo_reset_n.write(1)


In [2]:
import ipywidgets as widgets
import os

# Load Program GUI

This cell creates the GUI for eelecting and writing programs to the overlay.
It allows you to select between any ".bin" file found int he program folder (defaults to a folder called "programs" at same ;ocation fro this notebook).
This list of programs is created when this cell is tan, so when a new program being added to the program folder this file will have to be re-ran for it to appear in the dropdo2n list.

In [3]:
# Locations of program folder
program_folder  = os.path.join(".", "programs")

# GUI Widgets 
load_program_box = widgets.HBox()

load_program_select_dropdown_label = widgets.Label(
    value="Program select:"
)
load_program_select_dropdown = widgets.Dropdown(
    options=[
        f for f in os.listdir(program_folder)
        if os.path.isfile(os.path.join(program_folder, f)) and f.endswith(".bin")
    ],
    disabled=False,
)
load_program_box.children=load_program_box.children + (load_program_select_dropdown_label, load_program_select_dropdown)


load_program_write_button = widgets.Button(
    description="Write",
    disabled=False,
    tooltip="Loads the selecting program into the overlay's BRAM"
)
load_program_box.children=load_program_box.children + (load_program_write_button,)


load_program_current_progrom_label = widgets.Label(
    value="Loaded:",    
)
load_program_current_progrom_display = widgets.Label(
    value="",    
)
load_program_box.children=load_program_box.children + (load_program_current_progrom_label, load_program_current_progrom_display)

# Functions to handle interacting with the widgets, to load a program into the overlay's BRAM
def load_program_on_click(_):
    program = load_program_select_dropdown.value
    load_program_current_progrom_display.value = program
    write_program(os.path.join(program_folder, program))
    
def write_program(program_name):
    BRAM_data = [] 
    with open(program_name, "rb") as f:
        addr = 0
        raw_bytes = raw_bytes = f.read(4)

        while raw_bytes != b"":
            BRAM_data.append(
                sum([
                    raw_bytes[3] << 3*8,
                    raw_bytes[2] << 2*8,
                    raw_bytes[1] << 1*8,
                    raw_bytes[0] << 0*8,
                ])
            )

            addr += 4
            raw_bytes = f.read(4)

    neo_reset_n.write(0)

    for addr, word in enumerate(BRAM_data):
        neo_overlay.pynq_BRAM_controller.write(4*addr, word)

    neo_reset_n.write(1)
   
load_program_write_button.on_click(load_program_on_click)

# Finish on the enclosing box to display the GUI
load_program_box


# Overlay Stepper GUI

This cell creates the GUI for controlling the clock stepper of for the overlay's NEORV core.
Letting you enable/disable and configure each of the supposed stepper rules, before sending them to the overlay

In [4]:
# GUI Widgets 
stepper_box = widgets.VBox()

## first last rule
first_last_rule_box = widgets.HBox() 

first_last_rule_label = widgets.Label(
    value="First Last Rule:"
)
first_last_rule_toggle = widgets.ToggleButtons(
    options=["FIRST", "ALL"], 
    value = "FIRST",
    disabled=False,
    tooltips=[
        "The stepper will stop the NEORV core as soon at the first rule is fulfilled",
        "The stepper will onlt stop the NEORV core once all the rules for been fulfilled"
    ],
)
first_last_rule_box.children=(first_last_rule_label, first_last_rule_toggle)
stepper_box.children=stepper_box.children + (first_last_rule_box,)


## continous run rule
continous_run_rule_box = widgets.HBox()
continous_run_rule_label = widgets.Label(
    value="Continous Run Rule:"
)
continous_run_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
continous_run_rule_box.children=(continous_run_rule_label, continous_run_rule_enable,)
stepper_box.children=stepper_box.children + (continous_run_rule_box,)


## clock tick rule
clock_tick_rule_box = widgets.HBox()
clock_tick_rule_label = widgets.Label(
    value="Clock Tick Rule:"
)
clock_tick_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
clock_tick_rule_box.children=(clock_tick_rule_label, clock_tick_rule_enable,)
stepper_box.children=stepper_box.children + (clock_tick_rule_box,)


## clock count rule
clock_count_rule_box = widgets.HBox()
clock_count_rule_label = widgets.Label(
    value="Clock count Rule:"
)
clock_count_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
clock_count_rule_count_label = widgets.Label(
    value="Count:"
)
clock_count_rule_count_value = widgets.BoundedIntText(
    value=0,
    min=0,
    max=2**32,
    step=1,
    description="",
    disabled=False
)
clock_count_rule_box.children=(clock_count_rule_label, clock_count_rule_enable, clock_count_rule_count_label, clock_count_rule_count_value,)
stepper_box.children=stepper_box.children + (clock_count_rule_box,)


## Instr tick rule
instr_tick_rule_box = widgets.HBox()
instr_tick_rule_label = widgets.Label(
    value="Instr Tick Rule:"
)
instr_tick_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
instr_tick_rule_box.children=(instr_tick_rule_label, instr_tick_rule_enable,)
stepper_box.children=stepper_box.children + (instr_tick_rule_box,)


## Instr count rule
instr_count_rule_box = widgets.HBox()
instr_count_rule_label = widgets.Label(
    value="Instr count Rule:"
)
instr_count_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
instr_count_rule_count_label = widgets.Label(
    value="Count:"
)
instr_count_rule_count_value = widgets.BoundedIntText(
    value=0,
    min=0,
    max=2**32,
    step=1,
    description="",
    disabled=False
)
instr_count_rule_box.children=(instr_count_rule_label, instr_count_rule_enable, instr_count_rule_count_label, instr_count_rule_count_value,)
stepper_box.children=stepper_box.children + (instr_count_rule_box,)


## PC target rule
PC_target_rule_box = widgets.HBox()
PC_target_rule_label = widgets.Label(
    value="PC Target Rule:"
)
PC_target_rule_enable = widgets.Checkbox(
    value=False,
    description="",
    disabled=False,
    indent=False
)
PC_target_rule_count_label = widgets.Label(
    value="Count:"
)
PC_target_rule_count_value = widgets.BoundedIntText(
    value=0,
    min=0,
    max=64**10,
    step=1,
    description="",
    disabled=False
)
PC_target_rule_box.children=(PC_target_rule_label, PC_target_rule_enable, PC_target_rule_count_label, PC_target_rule_count_value,)
stepper_box.children=stepper_box.children + (PC_target_rule_box,)


## Run stepper Button
run_sweeper_box = widgets.HBox()
run_sweeper_button = widgets.Button(
    description="Run",
    disabled=False,
    tooltip="Push the currect setting of each of the rules to the overlay's stepper"
)
run_sweeper_box.children = (run_sweeper_button,)
stepper_box.children=stepper_box.children + (run_sweeper_box,)


# Functions to handle interacting with the widgets, to cintrol the stepper
def control_stepper(_):
    # Names for stepper's registors
    register_controls    = 0x0
    register_clock_count = 0x4
    register_instr_count = 0x8
    register_PC_targer   = 0xC
    
    # Names for the bits for the control registor
    control_bit_run_all     = 0x00000001
    control_bit_continous   = 0x00000100
    control_bit_clock_tick  = 0x00010000
    control_bit_clock_count = 0x00020000
    control_bit_instr_tick  = 0x01000000
    control_bit_instr_count = 0x02000000
    control_bit_PC_targer   = 0x04000000
        
    # First stop the stepper
    neo_overlay.cpu_clock_stepper.write(register_controls, 0)
    
    # Start wuth all rules disabled
    control_bits = 0
    
    # Handle first_last_rule
    if first_last_rule_toggle.value == "ALL":
        control_bits |= control_bit_run_all
    elif first_last_rule_toggle.value == "FIRST":
        pass
    else:
        raise ValueError("Unknown first_last_rule_toggle.value, " + str(first_last_rule_toggle.value))
     
    
    # Handle continous_run_rule
    if continous_run_rule_enable.value == True:
        control_bits |= control_bit_continous
    
    
    # Handle clock_tick_rule
    if clock_tick_rule_enable.value == True:
        control_bits |= control_bit_clock_tick
        
    # Handle clock_count_rule
    if clock_count_rule_enable.value == True and clock_count_rule_count_value.value != 0:
        neo_overlay.cpu_clock_stepper.write(register_clock_count, clock_count_rule_count_value.value)
        control_bits |= control_bit_clock_count
        
        
    # Handle instr_tick_rule
    if instr_tick_rule_enable.value == True:
        control_bits |= control_bit_instr_tick
        
    # Handle register_instr_count
    if instr_count_rule_enable.value == True and instr_count_rule_count_value.value != 0:
        neo_overlay.cpu_clock_stepper.write(register_instr_count, instr_count_rule_count_value.value)
        control_bits |= control_bit_instr_count
        
    # Handle register_clock_count
    if PC_target_rule_enable.value == True:
        neo_overlay.cpu_clock_stepper.write(register_PC_targer, 0x40000000 + PC_target_rule_count_value.value) 
        control_bits |= control_bit_PC_targer
    
    # Write controls to stepper
    neo_overlay.cpu_clock_stepper.write(register_controls, control_bits)
        
run_sweeper_button.on_click(control_stepper)


# Finish on the enclosing box to display the GUI
stepper_box

# Overlay Capture GUI

This cell creates the GUI displaying the captured signal of the overlay

In [ ]:
# Common update behavours
def update_recurse(this, capture_data):
    for child in this.children:
        child.update(child, capture_data)

def update_paas(this, capture_data):
    pass

def update_hex_word(this, capture_data):
    this.value = ("0"*8 + (hex(capture_data[this.capture_word])[2:]))[-8: ]

def update_hex_bits(this, capture_data):
    word = ("0"*32 + (bin(capture_data[this.capture_word])[2:]))[-32: ]
    bits = word[this.capture_bits_start:this.capture_bits_start + this.capture_bits_len]
    this.value = hex(int(bits, 2))[2:]

def update_bits_LUT(this, capture_data):
    word = ("0"*32 + (bin(capture_data[this.capture_word])[2:]))[-32: ]
    bits = word[this.capture_bits_start:this.capture_bits_start + this.capture_bits_len]
    this.value = this.capture_LUT[int(bits, 2)]

disale_enanble_LUT = ["disabled", "enabled"]

# GUI Widgets
capture_box = []

## regfile section
regfile_box = []

reg_symname_LUT = [
    "Zero", "ra",   "sp",   "gp",
    "tp",   "t0",   "t1",   "t1",
    "s0/fp",    "s1",   "a0",   "a1",
    "a2",   "a3",   "a4",   "a5",
    "a6",   "a7",   "s2",    "s3",
    "s4",   "s5",   "s6",    "s7",
    "s8",   "s9",   "s10",   "s11",
    "t3",   "t4",   "t5",    "t6",
]

### regfile rs1 section
regfile_rs1_box = []

#### regfile rs1 section label
regfile_rs1_box_label = widgets.Label("Source Register 1")
regfile_rs1_box_label.update = update_paas
regfile_rs1_box.append(regfile_rs1_box_label)

#### regfile rs1 section Addr
regfile_rs1_addr_line = []

regfile_rs1_addr_label = widgets.Label("rs1 addr :")
regfile_rs1_addr_label.update = update_paas
regfile_rs1_addr_line.append(regfile_rs1_addr_label)

regfile_rs1_addr_value = widgets.Label()
regfile_rs1_addr_value.update = update_hex_bits
regfile_rs1_addr_value.capture_word = 14
regfile_rs1_addr_value.capture_bits_start = 0
regfile_rs1_addr_value.capture_bits_len = 5
regfile_rs1_addr_line.append(regfile_rs1_addr_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs1_addr_line = widgets.HBox(regfile_rs1_addr_line)
regfile_rs1_addr_line.update = update_recurse
regfile_rs1_box.append(regfile_rs1_addr_line)

#### regfile rs1 section symbolic name
regfile_rs1_name_line = []

regfile_rs1_name_label = widgets.Label("rs1 name :")
regfile_rs1_name_label.update = update_paas
regfile_rs1_name_line.append(regfile_rs1_name_label)

regfile_rs1_name_value = widgets.Label()
regfile_rs1_name_value.update = update_bits_LUT
regfile_rs1_name_value.capture_word = 14
regfile_rs1_name_value.capture_bits_start = 0
regfile_rs1_name_value.capture_bits_len = 5
regfile_rs1_name_value.capture_LUT = reg_symname_LUT
regfile_rs1_name_line.append(regfile_rs1_name_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs1_name_line = widgets.HBox(regfile_rs1_name_line)
regfile_rs1_name_line.update = update_recurse
regfile_rs1_box.append(regfile_rs1_name_line)

#### regfile rs1 section output value
regfile_rs1_output_line = []

regfile_rs1_output_label = widgets.Label("rs1 out :")
regfile_rs1_output_label.update = update_paas
regfile_rs1_output_line.append(regfile_rs1_output_label)

regfile_rs1_output_value = widgets.Label()
regfile_rs1_output_value.update = update_hex_word
regfile_rs1_output_value.capture_word = 15
regfile_rs1_output_line.append(regfile_rs1_output_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs1_output_line = widgets.HBox(regfile_rs1_output_line)
regfile_rs1_output_line.update = update_recurse
regfile_rs1_box.append(regfile_rs1_output_line)

### Convert child list to widget box, then add box to to partner list
regfile_rs1_box = widgets.VBox(regfile_rs1_box)
regfile_rs1_box.update = update_recurse
regfile_box.append(regfile_rs1_box)

### regfile rs2 section
regfile_rs2_box = []

#### regfile rs2 section label
regfile_rs2_box_label = widgets.Label("Source Register 2")
regfile_rs2_box_label.update = update_paas
regfile_rs2_box.append(regfile_rs2_box_label)

#### regfile rs2 section Addr
regfile_rs2_addr_line = []

regfile_rs2_addr_label = widgets.Label("rs2 addr :")
regfile_rs2_addr_label.update = update_paas
regfile_rs2_addr_line.append(regfile_rs2_addr_label)

regfile_rs2_addr_value = widgets.Label()
regfile_rs2_addr_value.update = update_hex_bits
regfile_rs2_addr_value.capture_word = 14
regfile_rs2_addr_value.capture_bits_start = 8
regfile_rs2_addr_value.capture_bits_len = 5
regfile_rs2_addr_line.append(regfile_rs2_addr_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs2_addr_line = widgets.HBox(regfile_rs2_addr_line)
regfile_rs2_addr_line.update = update_recurse
regfile_rs2_box.append(regfile_rs2_addr_line)

#### regfile rs2 section symbolic name
regfile_rs2_name_line = []

regfile_rs2_name_label = widgets.Label("rs2 name :")
regfile_rs2_name_label.update = update_paas
regfile_rs2_name_line.append(regfile_rs2_name_label)

regfile_rs2_name_value = widgets.Label()
regfile_rs2_name_value.update = update_bits_LUT
regfile_rs2_name_value.capture_word = 14
regfile_rs2_name_value.capture_bits_start = 8
regfile_rs2_name_value.capture_bits_len = 5
regfile_rs2_name_value.capture_LUT = reg_symname_LUT
regfile_rs2_name_line.append(regfile_rs2_name_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs2_name_line = widgets.HBox(regfile_rs2_name_line)
regfile_rs2_name_line.update = update_recurse
regfile_rs2_box.append(regfile_rs2_name_line)

#### regfile rs2 section output value
regfile_rs2_output_line = []

regfile_rs2_output_label = widgets.Label("rs2 out :")
regfile_rs2_output_label.update = update_paas
regfile_rs2_output_line.append(regfile_rs2_output_label)

regfile_rs2_output_value = widgets.Label()
regfile_rs2_output_value.update = update_hex_word
regfile_rs2_output_value.capture_word = 16
regfile_rs2_output_line.append(regfile_rs2_output_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rs2_output_line = widgets.HBox(regfile_rs2_output_line)
regfile_rs2_output_line.update = update_recurse
regfile_rs2_box.append(regfile_rs2_output_line)

### Convert child list to widget box, then add box to to partner list
regfile_rs2_box = widgets.VBox(regfile_rs2_box)
regfile_rs2_box.update = update_recurse
regfile_box.append(regfile_rs2_box)

### regfile rd section
regfile_rd_box = []

#### regfile rd section label
regfile_rd_box_label = widgets.Label("Destination Register")
regfile_rd_box_label.update = update_paas
regfile_rd_box.append(regfile_rd_box_label)

#### regfile rd section writeback enable
regfile_rd_writeback_line = []

regfile_rd_writeback_label = widgets.Label("Regfile write :")
regfile_rd_writeback_label.update = update_paas
regfile_rd_writeback_line.append(regfile_rd_writeback_label)

regfile_rd_writeback_value = widgets.Label()
regfile_rd_writeback_value.update = update_bits_LUT
regfile_rd_writeback_value.capture_word = 14
regfile_rd_writeback_value.capture_bits_start = 16
regfile_rd_writeback_value.capture_bits_len = 1
regfile_rd_writeback_value.capture_LUT = disale_enanble_LUT
regfile_rd_writeback_line.append(regfile_rd_writeback_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_writeback_line = widgets.HBox(regfile_rd_writeback_line)
regfile_rd_writeback_line.update = update_recurse
regfile_rd_box.append(regfile_rd_writeback_line)

#### regfile rd section force_zero enable
regfile_rd_force_zero_line = []
regfile_rd_force_zero_label = widgets.Label("Regfile force addr 0 :")
regfile_rd_force_zero_label.update = update_paas
regfile_rd_force_zero_line.append(regfile_rd_force_zero_label)

regfile_rd_force_zero_value = widgets.Label()
regfile_rd_force_zero_value.update = update_bits_LUT
regfile_rd_force_zero_value.capture_word = 14
regfile_rd_force_zero_value.capture_bits_start = 17
regfile_rd_force_zero_value.capture_bits_len = 1
regfile_rd_force_zero_value.capture_LUT = disale_enanble_LUT
regfile_rd_force_zero_line.append(regfile_rd_force_zero_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_force_zero_line = widgets.HBox(regfile_rd_force_zero_line)
regfile_rd_force_zero_line.update = update_recurse
regfile_rd_box.append(regfile_rd_force_zero_line)

#### regfile rd section Addr
regfile_rd_addr_line = []

regfile_rd_addr_label = widgets.Label("rd addr :")
regfile_rd_addr_label.update = update_paas
regfile_rd_addr_line.append(regfile_rd_addr_label)

regfile_rd_addr_value = widgets.Label()
regfile_rd_addr_value.update = update_hex_bits
regfile_rd_addr_value.capture_word = 14
regfile_rd_addr_value.capture_bits_start = 16
regfile_rd_addr_value.capture_bits_len = 5
regfile_rd_addr_line.append(regfile_rd_addr_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_addr_line = widgets.HBox(regfile_rd_addr_line)
regfile_rd_addr_line.update = update_recurse
regfile_rd_box.append(regfile_rd_addr_line)

#### regfile rd section symbolic name
regfile_rd_name_line = []

regfile_rd_name_label = widgets.Label("rd name :")
regfile_rd_name_label.update = update_paas
regfile_rd_name_line.append(regfile_rd_name_label)

regfile_rd_name_value = widgets.Label()
regfile_rd_name_value.update = update_bits_LUT
regfile_rd_name_value.capture_word = 14
regfile_rd_name_value.capture_bits_start = 16
regfile_rd_name_value.capture_bits_len = 5
regfile_rd_name_value.capture_LUT = reg_symname_LUT
regfile_rd_name_line.append(regfile_rd_name_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_name_line = widgets.HBox(regfile_rd_name_line)
regfile_rd_name_line.update = update_recurse
regfile_rd_box.append(regfile_rd_name_line)


#### regfile rd section mem input
regfile_rd_mem_input_line = []

regfile_rd_mem_input_label = widgets.Label("Memory input :")
regfile_rd_mem_input_label.update = update_paas
regfile_rd_mem_input_line.append(regfile_rd_mem_input_label)

regfile_rd_mem_input_value = widgets.Label()
regfile_rd_mem_input_value.update = update_hex_word
regfile_rd_mem_input_value.capture_word = 21
regfile_rd_mem_input_line.append(regfile_rd_mem_input_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_mem_input_line = widgets.HBox(regfile_rd_mem_input_line)
regfile_rd_mem_input_line.update = update_recurse
regfile_rd_box.append(regfile_rd_mem_input_line)

#### regfile rd section alu input
regfile_rd_alu_input_line = []

regfile_rd_alu_input_label = widgets.Label("ALU input :")
regfile_rd_alu_input_label.update = update_paas
regfile_rd_alu_input_line.append(regfile_rd_alu_input_label)

regfile_rd_alu_input_value = widgets.Label()
regfile_rd_alu_input_value.update = update_hex_word
regfile_rd_alu_input_value.capture_word = 12
regfile_rd_alu_input_line.append(regfile_rd_alu_input_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_alu_input_line = widgets.HBox(regfile_rd_alu_input_line)
regfile_rd_alu_input_line.update = update_recurse
regfile_rd_box.append(regfile_rd_alu_input_line)

#### regfile rd section input select
regfile_rd_input_select_line = []

regfile_rd_input_select_label = widgets.Label("ALU input :")
regfile_rd_input_select_label.update = update_paas
regfile_rd_input_select_line.append(regfile_rd_input_select_label)

regfile_rd_input_select_value = widgets.Label()
regfile_rd_input_select_value.update = update_bits_LUT
regfile_rd_input_select_value.capture_word = 12
regfile_rd_input_select_value.capture_bits_start = 28
regfile_rd_input_select_value.capture_bits_len = 1
regfile_rd_input_select_value.capture_LUT = ["mem", "ALU"]
regfile_rd_input_select_line.append(regfile_rd_input_select_value)

#### Convert child list to widget box, then add box to to partner list
regfile_rd_input_select_line = widgets.HBox(regfile_rd_input_select_line)
regfile_rd_input_select_line.update = update_recurse
regfile_rd_box.append(regfile_rd_input_select_line)

### Convert child list to widget box, then add box to to partner list
regfile_rd_box = widgets.VBox(regfile_rd_box)
regfile_rd_box.update = update_recurse
regfile_box.append(regfile_rd_box)

## Convert child list to widget box, then add box to to partner list
regfile_box = widgets.VBox(regfile_box)
regfile_box.update = update_recurse
capture_box.append(regfile_box)


## ALU section
alu_box = []

### ALU operand a section
alu_operand_A_box = []

#### ALU a section label
alu_operand_A_box_label = widgets.Label("Operand A")
alu_operand_A_box_label.update = update_paas
alu_operand_A_box.append(alu_operand_A_box_label)

#### ALU a section section rs1 input
alu_operand_A_rs1_input_line = []

alu_operand_A_rs1_input_label = widgets.Label("Source Reg 1 input :")
alu_operand_A_rs1_input_label.update = update_paas
alu_operand_A_rs1_input_line.append(alu_operand_A_rs1_input_label)

alu_operand_A_rs1_input_value = widgets.Label()
alu_operand_A_rs1_input_value.update = update_hex_word
alu_operand_A_rs1_input_value.capture_word = 15
alu_operand_A_rs1_input_line.append(alu_operand_A_rs1_input_value)

#### Convert child list to widget box, then add box to to partner list
alu_operand_A_rs1_input_line = widgets.HBox(alu_operand_A_rs1_input_line)
alu_operand_A_rs1_input_line.update = update_recurse
alu_operand_A_box.append(alu_operand_A_rs1_input_line)

#### regfile rd section alu input
alu_operand_A_PC_input_line = []

alu_operand_A_PC_input_label = widgets.Label("PC input :")
alu_operand_A_PC_input_label.update = update_paas
alu_operand_A_PC_input_line.append(alu_operand_A_PC_input_label)

alu_operand_A_PC_input_value = widgets.Label()
alu_operand_A_PC_input_value.update = update_hex_word
alu_operand_A_PC_input_value.capture_word = 2
alu_operand_A_PC_input_line.append(alu_operand_A_PC_input_value)

#### Convert child list to widget box, then add box to to partner list
alu_operand_A_PC_input_line = widgets.HBox(alu_operand_A_PC_input_line)
alu_operand_A_PC_input_line.update = update_recurse
alu_operand_A_box.append(alu_operand_A_PC_input_line)

#### regfile rd section input select
alu_operand_A_select_line = []

alu_operand_A_select_label = widgets.Label("ALU input :")
alu_operand_A_select_label.update = update_paas
alu_operand_A_select_line.append(alu_operand_A_select_label)

alu_operand_A_select_value = widgets.Label()
alu_operand_A_select_value.update = update_bits_LUT
alu_operand_A_select_value.capture_word = 0
alu_operand_A_select_value.capture_bits_start = 0
alu_operand_A_select_value.capture_bits_len = 1
alu_operand_A_select_value.capture_LUT = ["rd1", "PC"]
alu_operand_A_select_line.append(alu_operand_A_select_value)

#### Convert child list to widget box, then add box to to partner list
alu_operand_A_select_line = widgets.HBox(alu_operand_A_select_line)
alu_operand_A_select_line.update = update_recurse
alu_operand_A_box.append(alu_operand_A_select_line)

### Convert child list to widget box, then add box to to partner list
alu_operand_A_box = widgets.VBox(alu_operand_A_box)
alu_operand_A_box.update = update_recurse
alu_box.append(alu_operand_A_box)

## Convert child list to widget box, then add box to to partner list
alu_box = widgets.VBox(alu_box)
alu_box.update = update_recurse
capture_box.append(alu_box)

# Convert capture_box from list to widget
capture_box = widgets.VBox(capture_box)
capture_box.update = update_recurse

def capture_signals():
    captured_data = [
        neo_overlay.neo_signal_capture.read(4*addr)
        for addr in range(60)
    ]
    return captured_data

def update_capture_GUI():
    captured_data = capture_signals()
    capture_box.update(capture_box, captured_data)

# Finish on capture_box to display GUI
capture_box


In [ ]:
update_capture_GUI()